build_dataset 해당

In [ ]:
# from agents.macro_classes.macro_class_dataset import MacroAData
# 
# macro_agent = MacroAData()
# macro_agent.fetch_data()
# macro_agent.add_features()
# macro_agent.save_csv()
# macro_agent.make_close_price()

$$$$   X = macro_sercher(agent, ticker) 역할

In [ ]:
from agents.macro_agent import MacroPredictor
from datetime import datetime

macro_sub = MacroPredictor(
    agent_id='MacroSentiAgent',
    base_date=datetime.today(),
    window=40,
    ticker="NVDA"
)
macro_sub.load_assets()               # 모델, 스케일러 등 불러오기


In [ ]:
macro_sub.fetch_macro_data()          # macro_df 불러오기
X_seq, X_scaled = macro_sub.prepare_features()  # 입력 시퀀스 준비

In [ ]:
print(f"X_seq: {X_seq}")
print(f"X_scaled: {X_scaled}")

$$$$   macro_predictor(X) 역할

In [ ]:
pred_prices, target = macro_sub.m_predictor(X_seq)

In [ ]:
print(f"pred_prices: {pred_prices}")
print(f"target: {target}")

llm 생성 용.. macro_reviewer_draft

In [ ]:
 total_json, opinion = macro_sub.macro_reviewer_draft(X_scaled, pred_prices, target)
 

In [ ]:
print(f"total_json: {total_json}")
print(f"opinion: {opinion}")

reviewer_rebut

In [ ]:
# reviewer_rebut()

[토론 테스트]

In [1]:
from debate_ver4.agents_tmp.debate_agent import DebateAgent

debate_agent = DebateAgent(1, "NVDA")

C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


In [ ]:
debate_agent.run_dataset()

In [ ]:
# from agents.macro_classes.macro_funcs import macro_dataset
# # debate_agent.run_dataset() 에서 해당 코드가 실행안되는 경우
# #8분 정도 걸림
# macro_dataset('NVDA')

In [ ]:
for agent_id, agent in debate_agent.agents.items():
    scaler = getattr(agent, "scaler", None)
    y_scaler = getattr(scaler, "y_scaler", None)
    print(f"{agent_id}:", type(y_scaler), y_scaler)
    print(agent.ticker)
    # if agent_id =='MacroSentiAgent':
    #     print(agent.macro_df.columns[:20])


In [2]:
opinions = debate_agent.get_opinion(0, "NVDA")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


MacroSentiAgent의 데이터 로드.. macro_sercher
[INFO] 모델 및 스케일러 로드 중...
model_path: agents/models/NVDA_MacroSentiAgent.keras
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (44, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


...

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\shap\explainers\_kernel.py:708: UserWarning: Linear regression equation is singular, a least squares solutions is used instead.
To avoid this situation and get a regular matrix do one of the following:
1) turn up the number of samples,
2) turn up the L1 regularization with num_features(N) where N is less than the number of samples,
3) group features together to reduce the number of inputs that need to be explained.
  warnings.warn(


[Interaction SHAP] Computing correlation among 6 features...

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 189.73579396533802} =================

================= LLM Explanation =================
(1) Temporal 분석: 제공된 Temporal SHAP 데이터에서 대부분 변수들의 영향력이 0에 가깝거나 변하지 않아 최근 시점으로 갈수록 특정 변수의 영향력이 커졌다고 보기 어렵습니다. 다만, '^VIX_ret'와 '^TNX_ret'의 일부 시점에서 미미한 영향이 관찰되었으나 전반적으로 영향력은 약화 또는 미미한 상태입니다. 이는 시장 변동성(VIX)과 장기 금리(TNX)의 변동성이 최근 예측에 큰 영향을 미치지 않았음을 시사합니다. 금융적으로 이는 단기 변동성이나 금리 변화가 NVDA 주가 예측에 즉각적 영향을 주지 않았거나, 다른 요인들이 더 우세했음을 의미합니다.

(2) Causal 분석: 인과 효과가 큰 변수들은 주로 미국 주요 지수 및 원자재 가격 관련입니다. 'Open_GC=F'(금 선물 시가, +0.0725)와 'Open_QQQ'(나스닥 ETF 시가, +0.0601)가 주가 상승 요인으로 작용했습니다. 반면 'Close_GC=F'(-0.0503), 'Low_^GSPC'(-0.0324), 'Close_^DJI'(-0.0294) 등은 하락 요인입니다. 이는 금 가격 상승과 기술주 중심의 나스닥 상승이 NVDA 주가에 긍정적 영향을 주었고, 반대로 S&P500과 다우지수의 일부 하락 신호가 부정적 영향을 미쳤음을 나타냅니다. 인과 구조로는 "금 가격 상승(안전자산 선호) 및 나스닥 강세(기술주 수요 증가) → NVDA 주가 상승"으로 해석할 수 있습니다.

(3) Interaction 분석: 상호작용이 높은 변수 쌍은 'Low_^IX

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


 Round 0 의견 수집 완료 (3 agents)


In [3]:
print(f"opinions:{opinions}")

opinions:{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=100.27559325248694, uncertainty=0.13370323181152344, confidence=7.479249954223633, feature_cols=None, importances=None), reason='현재가 207.04 대비 약 1.4% 상승한 210.0을 예상합니다. 최근 3년간 매출과 이익이 꾸준히 증가하는 추세이며, 마진도 안정적으로 유지되고 있습니다. 현금흐름이 긍정적이며, 밸류에이션 측면에서 현재 주가는 성장 잠재력 대비 다소 저평가되어 있습니다. 따라서 단기적으로 긍정적인 주가 상승이 기대됩니다.'), 'MacroSentiAgent': Opinion(agent_id='MacroSentiAgent', target=Target(next_close=189.7358, uncertainty=0.07873515039682388, confidence=12.7008, feature_cols=None, importances=None), reason='1. 기술적 분석 측면에서 NVDA의 5일 이동평균선(186.1)과 10일 이동평균선(184.08)이 현재 가격 대비 낮은 수준에 위치해 있어 단기 상승 모멘텀이 존재함을 시사합니다. 거래량은 평균적으로 안정적인 수준이며, RSI 등의 과매수 신호는 나타나지 않아 추가 상승 여력이 있습니다. 2. 거시경제 변수로는 금리 지표인 ^TNX(10년물 국채금리)가 4.0% 내외로 안정적이며, 인플레이션 압력 완화 기대감이 반영되어 있습니다. 환율(USDJPY)은 152.4 수준으로 큰 변동성이 없으며, 유동성 측면에서 주요 지수인 SPY, QQQ의 거래량이 충분히 유지되고 있어 시장 전반의 자금 흐름이 원활합니다. 3. 시장심리 및 정책 방향은 VIX 지수가 16.56으로 비교적 낮은 변동성을 나타내며, 이는 투자자들의 위험 회피 심리가 완화

리뷰탈 해보기..

In [4]:
debate_agent.get_rebuttal(1)

 Round 1 rebuttals 생성 완료 (6 agents)


[Rebuttal(from_agent_id='TechnicalAgent', to_agent_id='MacroSentiAgent', stance='REBUT', message='상대 에이전트는 189.74라는 보수적인 상승을 예측하며, 기술적 지표와 거시경제 환경을 근거로 들었으나, 이는 현재가 207.04 대비 크게 낮은 수치입니다. 5일 및 10일 이동평균선이 낮은 위치에 있다는 점은 단기 상승 모멘텀을 시사하지만, 이는 강한 상승 신호로 해석되어야 하며, 1.4 상승 예상은 과소평가된 결과입니다. RSI가 과매수 상태가 아니고 거래량이 안정적이라는 점은 추가 상승 여력을 명확히 보여주므로, 더 대담한 상승 목표가 타당합니다. 따라서 상대의 보수적 예측은 기술적 신호와 모멘텀 분석에 부합하지 않으며, 내 예측을 지지하는 것이 합리적입니다.'),
 Rebuttal(from_agent_id='TechnicalAgent', to_agent_id='SentimentalAgent', stance='REBUT', message='상대 에이전트의 52.7 상승 예측은 지나치게 낙관적이며, 기술적 분석 관점에서 모멘텀과 거래량 신호가 이를 뒷받침하지 않습니다. RSI나 추세 지표가 과매수 상태를 시사할 경우, 급격한 상승보다는 조정 가능성이 높습니다. 반면, 본인의 1.4 상승 예측은 안정적인 펀더멘털과 밸류에이션을 기반으로 한 보수적 접근으로, 기술적 신호와도 일관성이 있습니다. 따라서 상대의 과도한 상승 기대는 현실적이지 않으며, 단기 변동성 확대보다는 점진적 상승을 예상하는 것이 합리적입니다.'),
 Rebuttal(from_agent_id='MacroSentiAgent', to_agent_id='TechnicalAgent', stance='REBUT', message='상대의 의견은 NVDA 주가가 현재가 대비 소폭 상승할 것이라는 점에서 긍정적이나, 제시한 예상 종가가 현재가보다 크게 낮아 현실과 부합하지 않습니다. 최근 거시경제 변수인 금리 안정, 인플레이션 완

get_revise 해보기

In [ ]:
debate_agent.get_revise(1)

'''
에러 나오는 중..
[TechnicalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
[SentimentalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
'''

결과 도출해보기

In [ ]:
debate_agent.get_ensemble()

통합해서 한번에 실행하기: 기본 라운드는 총 3회. 클래스 정의떄 수정 가능

In [ ]:
#debate_agent.run()